In [ ]:
project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)

from google.cloud import bigquery
import importlib
import plotly.express as px

import numpy as np
import pandas as pd
from datetime import datetime

import ft_digital_trade.src.utils.read_data as read_utils
import ft_digital_trade.src.utils.clean_utils as clean_utils
import ft_digital_trade.src.utils.calculation_utils as calc_utils
import ft_digital_trade.src.utils.plot_utils as plot_utils

In [ ]:
client = bigquery.Client()

In [ ]:
#This allows you to select different location of merchant spend for face-to-face and online
## for more information run the following:
##help(read_utils.read_visa)
#visa_data = read_utils.read_f2f_online(cardholder_origin = "all", f2f =  "uk", online =  "all")

#This applies the same filters for face-to-face and online data
## for more information run the following:
##help(read_utils.read_visa)
visa_data = read_utils.read_visa(cardholder_origin = "uk", cardholders_location = "uk", spend_location = "uk")

In [ ]:
visa = calc_utils.calculate_visa(visa_data)
visa = clean_utils.rename_columns(df = visa, suffix = '_spoc')

In [ ]:
global_cards = read_utils.read_global_cards()
global_cards = clean_utils.clean_global(global_cards)
global_cards = calc_utils.calculate_global(global_cards, 'card')

In [ ]:
global_spend = read_utils.read_global_spend()
global_spend = clean_utils.clean_global(global_spend)
global_spend = calc_utils.calculate_global(global_spend, 'spend')

In [ ]:
global_df = global_cards.merge(global_spend, how = 'inner', on = 'year', suffixes = ('_cards', '_spend'))
global_df = clean_utils.rename_columns(df = global_df, suffix = '_global')

In [ ]:
uk_finance = read_utils.read_uk_finance()
uk_finance = clean_utils.clean_uk_finance(uk_finance)
uk_finance = calc_utils.calculate_uk_finance(uk_finance)
uk_finance = uk_finance[['year', 'cardholders','total value of purchases',"total volume of purchases"]]
uk_finance = clean_utils.rename_columns(df = uk_finance , suffix = '_uk_finance')

In [ ]:
boe = read_utils.read_boe()
boe = clean_utils.clean_boe(boe)
boe = calc_utils.calculate_boe(boe)
boe = clean_utils.rename_columns(df = boe , suffix = '_boe')

In [ ]:
link = read_utils.read_link()

In [ ]:
merged = visa.merge(uk_finance, how = 'outer', on = 'year')
merged = merged.merge(boe, how = 'outer', on = 'year')
merged = merged.merge(global_df, how = 'outer', on = 'year')

In [ ]:
cardholders = merged[['year','cardholders_spoc','cardholders_uk_finance','visa_total_cards_global','total_cards_global', 'visa_marketshare_cards_global']]
cardholders = cardholders.copy()
cardholders['uk_finance_marketshare'] = cardholders['cardholders_spoc'] / cardholders['cardholders_uk_finance'] *100
cardholders['global_marketshare'] = cardholders['cardholders_spoc'] / cardholders['total_cards_global'] *100
#melt df for charts
cardholders = pd.melt(cardholders, id_vars='year',var_name='Data source', value_name='value')
cardholders = calc_utils.calculate_index(df = cardholders)

In [ ]:
spend = merged[['year','spend_spoc', 
        'total value of purchases_uk_finance',
       'Mastercard values_boe', 'Visa Europe values_boe',
       'Mastercard and Visa values_boe', 'Visa proportion_boe',
       'debit_spend_global', 'credit_spend_global', 'visa_total_spend_global',
       'total_spend_global', 'visa_marketshare_spend_global']]
spend = spend.copy()
# #replace 2024 spending with NA
spend['spend_spoc'] = np.where(spend['year']==2024, np.nan, spend['spend_spoc'])
spend['total value of purchases_uk_finance'] = np.where(spend['year']==2024, np.nan, spend['total value of purchases_uk_finance'])
#calculate marketshare
spend['uk_finance_marketshare'] = spend['spend_spoc'] / spend['total value of purchases_uk_finance'] *100
spend['global_marketshare'] = spend['spend_spoc'] / spend['total_spend_global'] *100
spend['boe_marketshare'] = spend['spend_spoc'] / spend['Mastercard and Visa values_boe'] *100
#copy used for getting 2019 marketshare
spend_copy = spend.copy()
#melt df for charts
spend = pd.melt(spend, id_vars='year',var_name='Data source', value_name='value')
spend = calc_utils.calculate_index(df = spend)

In [ ]:
plot_utils.plot_total_cardholders(df = cardholders)

In [ ]:
plot_utils.plot_visa_cardholders(df = cardholders)

In [ ]:
plot_utils.plot_marketshare_cardholders(df = cardholders)

In [ ]:
plot_utils.plot_total_spend(df = spend)

In [ ]:
plot_utils.plot_visa_spend(df = spend)

In [ ]:
plot_utils.plot_marketshare_spend(df = spend)

In [ ]:
#options of marketshare threshold
# marketshare_2019 = spend_copy.iloc[0]['visa_marketshare_spend_global']
# marketshare_2019 = spend_copy.iloc[0]['global_marketshare']
marketshare_2019 = spend_copy.iloc[0]['uk_finance_marketshare']
marketshare_2019

In [ ]:
df = merged.copy()
#remove 2024 due to incomplete data
df = df[df['year'] != 2024]
#index spoc data
df['idx_cardholders_spoc'] = df['cardholders_spoc'].transform(lambda x: (x / x.iloc[0] * 100))
df['idx_spend_spoc'] = df['spend_spoc'].transform(lambda x: (x / x.iloc[0] * 100))
# adjust visa spend to 2019 cardholders
df['visa_adj_spend_spoc'] = (df['spend_spoc']/df['idx_cardholders_spoc'])*100
df['total_spoc'] = df['visa_adj_spend_spoc'] / marketshare_2019 *100
#rename columns
df = df.rename(columns={'total value of purchases_uk_finance': 'total_uk_finance'})
df = df.rename(columns={'visa_total_spend_global': 'visa_total_global','total_spend_global':'total_global'})
df = df.rename(columns={'Visa Europe values_boe': 'visa_total_boe', 'Mastercard and Visa values_boe': 'total_boe'})
#filter columns
df = df[['year', 'visa_adj_spend_spoc', 'total_spoc', 'visa_total_global', 'total_global', 'total_uk_finance', 'visa_total_boe', 'total_boe' ]]
#melt df
df = pd.melt(df, id_vars='year',var_name='Data source', value_name='value')
df = calc_utils.calculate_index(df = df)

In [ ]:
plot_utils.plot_adjusted_visa(df = df, y_value = "value")

In [ ]:
plot_utils.plot_adjusted_visa(df = df, y_value = "index")